# Imports

In [10]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from bs4 import BeautifulSoup
import time

options = Options()
# options.headless = True
driver = webdriver.Chrome(executable_path="faq_data/chromedriver.exe", options=options)

In [14]:
help(driver)

Help on WebDriver in module selenium.webdriver.chrome.webdriver object:

class WebDriver(selenium.webdriver.remote.webdriver.WebDriver)
 |  WebDriver(executable_path='chromedriver', port=0, options=None, service_args=None, desired_capabilities=None, service_log_path=None, chrome_options=None, keep_alive=True)
 |  
 |  Controls the ChromeDriver and allows you to drive the browser.
 |  
 |  You will need to download the ChromeDriver executable from
 |  http://chromedriver.storage.googleapis.com/index.html
 |  
 |  Method resolution order:
 |      WebDriver
 |      selenium.webdriver.remote.webdriver.WebDriver
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, executable_path='chromedriver', port=0, options=None, service_args=None, desired_capabilities=None, service_log_path=None, chrome_options=None, keep_alive=True)
 |      Creates a new instance of the chrome driver.
 |      
 |      Starts the service and then creates new instance of chrome driver.
 |      

In [11]:
driver.get("https://en.wikipedia.org/wiki/Phillip_Davey")

In [13]:
driver.find_element_by_xpath('//*[@id="firstHeading"]').text

'Phillip Davey'

In [4]:
with open("faq_data/disney_FAQ_URLs.json", 'r') as file:
    urls = json.load(file)

In [5]:
Areas = list(urls.keys())
Areas

['Disneyland Resort', 'Vacation Planning', 'Technical Support']

In [4]:
df = pd.DataFrame(columns=["area","section","question","answer"])

for Area in tqdm(Areas, desc="Areas"):
    sections = list(urls[Area].keys())
    for section in tqdm(sections, desc="Sections"):
        url = urls[Area][section]
        driver.get(url)
        
        # scroll down to element
        l= driver.find_element_by_xpath('//*[@id="gde_footerLegalContainer"]/div[2]')
        driver.execute_script("arguments[0].scrollIntoView(true);", l)
        
        # wait for 10 seconds for the page to load completely
        time.sleep(10)
        
        # get the question links
        ul = driver.find_element_by_xpath('//*[@id="pageContent"]/div[2]/div[1]/div[1]/div[2]/ul')
        qa_urls = ul.find_elements_by_tag_name('li')
        qa_urls = [qa_url.find_element_by_tag_name('a').get_attribute('href') for qa_url in qa_urls]

        for qa_url in tqdm(qa_urls, desc="Questions"):
            try:
                # open the question
                driver.get(qa_url)
                question = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((
                                By.XPATH, 
                                '//*[@id="pageContent"]/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/p'
                                ))).text
                answer = driver.find_element_by_xpath('//*[@id="pageContent"]/div[2]/div[1]/div[1]/div[2]/div/div[2]/div[2]').text
                # add to df
                df.loc[len(df.index)] = [Area, section, question, answer] 
#                 print([Area, section, question])
                driver.back()
            except Exception as e:
                print(e)
                df.loc[len(df.index)] = ["None", "None", "None", "None"]
                continue
        backup_df = df
        backup_df.to_csv("faq_data/"+Area+"_"+section+".csv", index=False)
            
df.to_excel("faq_data/faqs.xlsx", index=False)

Sections:   0%|                                                                                                                                 | 0/10 [00:00<?, ?it/s]

Questions:   0%|                                                                                                                                | 0/47 [00:00<?, ?it/s]

Questions:   2%|██▌                                                                                                                     | 1/47 [00:07<05:24,  7.05s/it]

Questions:   4%|█████                                                                                                                   | 2/47 [00:16<05:43,  7.63s/it]

Questions:   6%|███████▋                                                                                                                | 3/47 [00:22<05:21,  7.30s/it]

Questions:   9%|██████████▏                                                                                                             | 4/47 [00:30<05:21

Message: 





Questions:  53%|███████████████████████████████████████████████████████████████▎                                                       | 25/47 [03:22<03:10,  8.67s/it]

Questions:  55%|█████████████████████████████████████████████████████████████████▊                                                     | 26/47 [03:28<02:47,  7.96s/it]

Questions:  57%|████████████████████████████████████████████████████████████████████▎                                                  | 27/47 [03:35<02:34,  7.72s/it]

Questions:  60%|██████████████████████████████████████████████████████████████████████▉                                                | 28/47 [03:42<02:19,  7.36s/it]

Questions:  62%|█████████████████████████████████████████████████████████████████████████▍                                             | 29/47 [03:48<02:05,  6.95s/it]

Questions:  64%|███████████████████████████████████████████████████████████████████████████▉                                           | 30/47 [03:54<01:

Message: 





Questions:  81%|████████████████████████████████████████████████████████████████████████████████████████████████▏                      | 38/47 [04:51<01:09,  7.76s/it]

Questions:  83%|██████████████████████████████████████████████████████████████████████████████████████████████████▋                    | 39/47 [04:57<00:58,  7.30s/it]

Questions:  85%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎                 | 40/47 [05:04<00:50,  7.20s/it]

Questions:  87%|███████████████████████████████████████████████████████████████████████████████████████████████████████▊               | 41/47 [05:12<00:43,  7.31s/it]

Questions:  89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▎            | 42/47 [05:18<00:34,  6.97s/it]

Questions:  91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▊          | 43/47 [05:24<00:

...



Questions:  40%|████████████████████████████████████████████████                                                                        | 4/10 [00:43<01:08, 11.45s/it]

Questions:  50%|████████████████████████████████████████████████████████████                                                            | 5/10 [00:50<00:50, 10.10s/it]

Questions:  60%|████████████████████████████████████████████████████████████████████████                                                | 6/10 [01:04<00:45, 11.39s/it]

Message: 





Questions:  70%|████████████████████████████████████████████████████████████████████████████████████                                    | 7/10 [01:19<00:37, 12.34s/it]

Message: 





Questions:  80%|████████████████████████████████████████████████████████████████████████████████████████████████                        | 8/10 [01:26<00:21, 10.60s/it]

Questions:  90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████            | 9/10 [01:34<00:09,  9.86s/it]

Questions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:41<00:00, 10.17s/it]

Areas: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [41:13<00:00, 824.53s/it]


In [6]:
df.tail(8)

,area,section,question,answer
284,None,None,None,None
285,Technical Support,Technical Help,Does disneyland.com have a policy on Internet ...,Yes. The best form of online safety begins at ...
286,Technical Support,Technical Help,Do you offer a version of disneyland.com for d...,Yes. Disneyland.com offers the following inter...
287,None,None,None,None
288,None,None,None,None
289,Technical Support,Technical Help,What web browser should I use when visiting di...,Disneyland.com is designed for optimal viewing...
290,Technical Support,Technical Help,How do I report a problem with disneyland.com?,Report a problem. We apologize for any difficu...
291,Technical Support,Technical Help,How do I find something specific on disneyland...,Looking for a specific piece of information ab...


In [5]:
backup_df = df
backup_df.tail()
backup_df.to_excel("faq_data/qna.xlsx", index=False)